In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.integrate import solve_ivp, quad
import numpy as np
from scipy.misc import derivative as dt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [53]:
γ = 1/2.4
fecha_inicio = pd.to_datetime("27-february-2020")
r = 0.7
inicio_red = 15
f = 0.9
q = 0.1356473594548552
ν = 5/12
η = 1/7
ϵ = 1/1.4
θ = 1/2.4
pa, pm, ps = 0.5, 0.476, 0.024
σ = 1/4.7
ξ = 1/4
τ = 1/5

In [3]:
ABT = pd.read_csv("D:/Edgar Trejo/Universidad/Proyecto/COVID19Model/data/cleandata/ABTrf.csv", dtype={0:str}, index_col = 0)

In [61]:
ABT.to_numpy()

array([[2.32346000e+05, 1.24820000e+04, 8.96000000e+02, ...,
        0.00000000e+00, 1.41769300e+01, 1.92654531e-05],
       [9.50900000e+03, 6.20410000e+05, 9.25100000e+03, ...,
        0.00000000e+00, 2.27452228e+01, 2.13501530e-05],
       [1.29800000e+03, 8.53100000e+03, 1.38682000e+05, ...,
        0.00000000e+00, 2.19371702e+01, 5.16490072e-05],
       ...,
       [2.36600000e+02, 2.36600000e+02, 0.00000000e+00, ...,
        4.65562734e+02, 1.01190307e+03, 1.38411025e-03],
       [2.53190000e+03, 7.23800000e+02, 1.21800000e+02, ...,
        3.63334717e+02, 6.64460249e+02, 3.66123441e-04],
       [3.19200000e+02, 5.22900000e+02, 3.04500000e+02, ...,
        1.10384338e+02, 1.83499744e+02, 1.30436731e-04]])

In [4]:
pob = pd.read_csv('https://raw.githubusercontent.com/edtrelo/COVID19Model/main/data/cleandata/pob_miZMVM.csv',
                 index_col = 0, dtype = {'cve_umun':str})

In [5]:
pob = pob['pop'].to_list()

In [62]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [74]:
train_X, test_X, train_Y, test_Y = train_test_split(ABT.to_numpy()[:, :-1], ABT.to_numpy()[:, -1], test_size = 0.25, random_state = 42)

In [75]:
rf.fit(train_X, train_Y)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [20]:
base = ABT[[1], :-1]

In [76]:
def λ(t, i, Ia, Im, Ip):
    a = np.zeros(99)
    a[:47*2] = ABT.iloc[i, :47*2]
    a[47*2] = t
    a[47*2 + 1] = i
    a[47*2 + 2] = Ia
    a[47*2 + 3] = f*Im
    a[47*2 + 4] = Ip
    
    if t > inicio_red:
        a[:47*2] = (1-r)*a[:47*2]
    
    return a.reshape((1, 99))

In [77]:
λ(10, 1, 0, 0, 0)

array([[9.50900e+03, 6.20410e+05, 9.25100e+03, 2.36930e+04, 1.44450e+04,
        1.25925e+05, 7.12940e+04, 5.83200e+03, 4.61820e+04, 3.37800e+04,
        1.27829e+05, 3.55430e+04, 8.60730e+04, 7.16040e+04, 3.30320e+04,
        1.90850e+04, 0.00000e+00, 1.83000e+02, 1.98000e+03, 1.51200e+03,
        6.15000e+02, 3.30200e+03, 2.78700e+03, 8.37700e+03, 2.04480e+04,
        3.99000e+02, 3.77000e+02, 2.85900e+03, 1.43500e+03, 7.75400e+03,
        1.46050e+04, 6.58000e+02, 3.59000e+02, 2.74800e+03, 2.73000e+03,
        7.52000e+02, 6.60000e+01, 4.15000e+02, 1.23000e+02, 1.19600e+03,
        2.96000e+02, 6.96800e+03, 7.87000e+02, 1.58700e+03, 9.21000e+02,
        1.87400e+03, 1.23100e+03, 1.24820e+04, 6.20410e+05, 8.53100e+03,
        2.59750e+04, 1.43720e+04, 1.53000e+05, 7.50280e+04, 5.04500e+03,
        5.88430e+04, 2.36110e+04, 1.24246e+05, 4.62610e+04, 7.56090e+04,
        8.39200e+04, 2.62750e+04, 2.03950e+04, 0.00000e+00, 2.41000e+02,
        2.29900e+03, 1.84200e+03, 5.54000e+02, 4.00

In [78]:
L = lambda t, i, Ia, Im, Ip: rf.predict(λ(t, i, Ia, Im, Ip))[0]

In [79]:
L(10, 1, 0, 0, 0)

-3.6179467196637175e-06

In [88]:
def Model(t, X):
    
    dXdt = np.zeros(47*8)
    
    S = lambda i: X[8*i + 0]
    E = lambda i: X[8*i + 1]
    Ip = lambda i: X[8*i + 2]
    Iss =  lambda i: X[8*i + 3]
    H = lambda i: X[8*i + 4]
    Ia = lambda i: X[8*i + 6]
    Im = lambda i: X[8*i + 7]
            
    for i in range(47):    
        # X_i; los componentes de cada ciudad
        # ecuación de los suceptibles para la ciudad i
        dXdt[8*i + 0] = -L(t, i, Ia(i), Im(i), Ip(i))*S(i)
        # ecuación de los expuestos para la ciudad i
        dXdt[8*i + 1] = -dXdt[8*i + 0] - ϵ*E(i)
        # ecuación para los pre-infectados
        dXdt[8*i + 2] = ϵ*E(i) - θ*Ip(i)
        # ecuación para los infectados severos
        dXdt[8*i + 3] = θ*ps*Ip(i) - σ*Iss(i)
        # ecuación para los hospitalizados
        dXdt[8*i + 4] = σ*Iss(i) - (1-q)*τ*H(i) - ξ*q*H(i)
        # ecuación de los difuntos
        dXdt[8*i + 5] = q*ξ*H(i)
        # ecuación de los asintomáticos
        dXdt[8*i + 6] = θ*pa*Ip(i) - γ*Ia(i)
        # ecuación de los mild sym
        dXdt[8*i + 7] = θ*pm*Ip(i) - γ*Im(i)

    return dXdt

In [89]:
x0df = pd.read_csv("https://raw.githubusercontent.com/edtrelo/COVID19Model/main/data/cleandata/x0.csv", index_col = 0)

In [90]:
x0 = []
indicesCasos = [3, 5, 34]

for i in range(47):
    if i in indicesCasos:
        x0.extend([pob[i]-x0df.iloc[i, 0]-x0df.iloc[i, 1], x0df.iloc[i, 1], x0df.iloc[i, 0], 0, 0, 0, 1, 0])
    else:
        x0.extend([pob[i]-x0df.iloc[i, 0]-x0df.iloc[i, 1], x0df.iloc[i, 1], x0df.iloc[i, 0], 0, 0, 0, 0, 0])

In [91]:
tdata = np.linspace(0, 93, 94) #22:38 
    
sol = solve_ivp(Model, (0, 94), x0, t_eval = tdata)

KeyboardInterrupt: 